In [9]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/Transformer/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model
from utils import token2class, DotDict

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# PART 1: Configuration

In [2]:
args = argparse.Namespace(
    data_type="claim",
    target_ipc="",
    pred_type="classification",
    use_pretrained_tokenizer=False,
    do_train=None,
    do_tune=None,
    n_folds=None,
    max_epochs=2,
    use_accelerator=None,
    do_save=False,
    n_gpus=4,
    light=True,
    eval_train_set=False)

config_file = "configs_light.json" if args.light else "configs.json"
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

regex_ipc = re.compile('[A-Z](?![\\D])')
if regex_ipc.match(configs.data.target_ipc) is None:
    configs.data.update({"target_ipc": "ALL"})
elif len(configs.data.target_ipc) > 5:
    configs.data.update({"target_ipc": configs.data.target_ipc[:4]})

data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "models")

if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device

    configs.train.update({"accelerator": accelerator})
    # configs.model.update({"use_accelerator": configs.train.use_accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
    else:
        device = torch.device('cpu')
        device_ids = []

configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "n_outputs": 1 if configs.data.pred_type=="regression" else 2,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_hidden = configs.model.d_hidden = None
    d_latent = None
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_hidden = configs.model.d_hidden
    d_latent = configs.model.n_enc_seq * configs.model.d_hidden
    key_components = {"data": ["target_ipc", "pred_type", "max_seq_len", "vocab_size"], "model": ["n_layers", "d_hidden", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    config_name = ""
    for key in key_components.keys():
        for component in key_components[key]:
            config_name += "["+str(configs[key][component])+component+"]"
    final_model_path = os.path.join(model_dir, f"[Final_model]{config_name}.ckpt")

configs.model.update({"d_latent": d_latent})
configs.train.update({"config_name": config_name,
                        "final_model_path": final_model_path})

# PART 2: Dataset setting

In [3]:
tstart = time.time()
dataset_config_name = "-".join([str(key)+"="+str(value) for (key,value) in configs.data.items() if key in org_config_keys["data"]])
dataset_path = os.path.join(data_dir, "pickled_dataset", "[tech_dataset]"+dataset_config_name+".pickle")
if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    tech_dataset = TechDataset(configs.data)
    with open(dataset_path, "wb") as f:
        tech_dataset.rawdata = None
        pickle.dump(tech_dataset, f)
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

configs.model.update({"tokenizer": tech_dataset.tokenizer,
                        "n_enc_vocab": tech_dataset.tokenizer.get_vocab_size(),
                        "n_dec_vocab": tech_dataset.tokenizer.get_vocab_size(),
                        "n_enc_seq": tech_dataset.max_seq_len,
                        "n_dec_seq": tech_dataset.max_seq_len,
                        "i_padding": tech_dataset.tokenizer.token_to_id("<PAD>")})
configs.model.update({"d_latent": configs.model.n_enc_seq * configs.model.d_hidden})

Load pickled dataset...
Pickled dataset loaded
0.1516 sec elapsed for loading patents for class [ALL]


# PART 3: Training

In [4]:
sampler = CVSampler(tech_dataset, n_folds=configs.train.n_folds, test_ratio=0.1, stratify=True)
cv_idx = sampler.get_idx_dict()
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

#Samples
Train: 55847, Validation: 13962, Test: 29919


## PART 3-1: Hyperparameter tuning

In [ ]:
if configs.train.do_tune:
    configs.train.update({"tuned_model_path": os.path.join(model_dir,"hparam_tuning")})
    optuna_obj = lambda trial: objective_cv(trial, dataset=tech_dataset, cv_idx=cv_idx, model_params=configs.model, train_params=configs.train)

    opt_sampler = TPESampler()
    study = optuna.create_study(direction='minimize')
    study.optimize(optuna_obj, n_trials=configs.train.n_trials, gc_after_trial=True)
    best_params = study.best_trial.params

    print(f"Best trial:\n  CrossEntropyLoss: {study.best_trial.value}\n  Params:")
    for k, v in best_params.items():
        print(f"    {k}: {v}")

    configs.train.update({k: v for k,v in best_params.items() if k in configs.train.keys()})
    configs.model.update({k: v for k,v in best_params.items() if k in configs.model.keys()})
    config_name = f"{configs.model.n_layers}layers_{configs.model.d_embedding}emb_{configs.model.d_hidden}hid_{configs.model.n_directions}direc_{np.round(configs.train.learning_rate, 4)}lr_{configs.train.batch_size}batch_{configs.train.max_epochs}ep"
    final_model_path = os.path.join(model_dir, f"[Final_model][{configs.data.target_ipc}]{config_name}.ckpt")

## PART 3-2: Dataset construction and model training

In [36]:
## Construct datasets
train_idx = cv_idx[0]['train']
val_idx = cv_idx[0]['val']
test_idx = cv_idx[0]['test']
whole_idx = np.concatenate([train_idx, val_idx])

train_dataset = Subset(tech_dataset, train_idx)
val_dataset = Subset(tech_dataset, val_idx)
test_dataset = Subset(tech_dataset, test_idx)
whole_dataset = Subset(tech_dataset, whole_idx)

train_loader = DataLoader(train_dataset, batch_size=configs.train.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=configs.train.batch_size if len(val_idx)>configs.train.batch_size else len(val_idx), shuffle=True, num_workers=4, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=configs.train.batch_size if len(test_idx)>configs.train.batch_size else len(test_idx), shuffle=False, num_workers=4)
whole_loader = DataLoader(whole_dataset, batch_size=configs.train.batch_size, shuffle=False, num_workers=4)

## Load best model or build model
final_model = build_model(configs.model, tokenizer=tech_dataset.tokenizer)
# x_input, _ = next(iter(train_loader))
# if re.search("^1.", torch.__version__) is not None:
#     print(pytorch_model_summary.summary(final_model.module, torch.zeros(x_input.shape, device=device, dtype=torch.long), torch.zeros(x_input.shape, device=device, dtype=torch.long), show_input=True, max_depth=None, show_parent_layers=True))
# else:
#     print("INFO: pytorch-model-summary does not support PyTorch 2.0, so just print model structure")
#     print(final_model)
#     torch._dynamo.config.verbose = True
#     torch._dynamo.config.suppress_errors = True

In [38]:
p = list(final_model.module.named_parameters())[1]

In [46]:
for p in final_model.module.decoder.pos_emb.named_parameters():
    print(p)

('weight', Parameter containing:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  5.3317e-01,  ...,  1.0000e+00,
          1.7783e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.0213e-01,  ...,  1.0000e+00,
          3.5566e-04,  1.0000e+00],
        ...,
        [ 4.5200e-01, -8.9202e-01, -9.9420e-01,  ...,  9.9678e-01,
          4.5153e-02,  9.9898e-01],
        [-5.0639e-01, -8.6230e-01, -8.9845e-01,  ...,  9.9675e-01,
          4.5331e-02,  9.9897e-01],
        [-9.9921e-01, -3.9791e-02, -5.2599e-01,  ...,  9.9672e-01,
          4.5508e-02,  9.9896e-01]], device='cuda:0', dtype=torch.float64))


In [48]:
for param in final_model.module.decoder.named_parameters():
    param[1].requires_grad = False
for param in final_model.module.decoder.pos_emb.parameters():
    param.requires_grad = False
for param in final_model.module.named_parameters():
    if not param[1].requires_grad:
        print(param[0])

encoder.pos_emb.weight
decoder.dec_emb.weight
decoder.pos_emb.weight
decoder.layers.0.dec_self_attn.W_Q.weight
decoder.layers.0.dec_self_attn.W_Q.bias
decoder.layers.0.dec_self_attn.W_K.weight
decoder.layers.0.dec_self_attn.W_K.bias
decoder.layers.0.dec_self_attn.W_V.weight
decoder.layers.0.dec_self_attn.W_V.bias
decoder.layers.0.dec_self_attn.linear.weight
decoder.layers.0.dec_self_attn.linear.bias
decoder.layers.0.layer_norm1.weight
decoder.layers.0.layer_norm1.bias
decoder.layers.0.dec_enc_attn.W_Q.weight
decoder.layers.0.dec_enc_attn.W_Q.bias
decoder.layers.0.dec_enc_attn.W_K.weight
decoder.layers.0.dec_enc_attn.W_K.bias
decoder.layers.0.dec_enc_attn.W_V.weight
decoder.layers.0.dec_enc_attn.W_V.bias
decoder.layers.0.dec_enc_attn.linear.weight
decoder.layers.0.dec_enc_attn.linear.bias
decoder.layers.0.layer_norm2.weight
decoder.layers.0.layer_norm2.bias
decoder.layers.0.pos_ffn.conv1.weight
decoder.layers.0.pos_ffn.conv1.bias
decoder.layers.0.pos_ffn.conv2.weight
decoder.layers.0.po

In [ ]:
with torch.no_grad():
    x_input, _ = tech_dataset.__getitem__(0)
    x_input = torch.tensor(x_input, device=device).unsqueeze(0)
    print(pytorch_model_summary.summary(final_model.module, torch.zeros(x_input.shape, device=device, dtype=torch.long), torch.zeros(x_input.shape, device=device, dtype=torch.long), show_input=True, max_depth=None, show_parent_layers=True))

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:170% !important; }</style>"))

In [ ]:
## Train model
if configs.train.do_tune:
    best_states = torch.load(os.path.join(configs.train.tuned_model_path,f"[HPARAM_TUNING]{study.best_trial.number}trial.ckpt"))
    converted_states = OrderedDict()
    for k, v in best_states.items():
        if 'module' not in k:
            k = 'module.'+k
        else:
            k = k.replace('features.module.', 'module.features.')
        converted_states[k] = v
    final_model.load_state_dict(converted_states)
else:
    final_model = train_model(final_model, train_loader, val_loader, configs.model, configs.train)
torch.save(final_model.state_dict(), final_model_path) # Finalize
torch.cuda.empty_cache()

## PART 3-3: Training evaluation

In [ ]:
if args.eval_train_set:
    ## Evaluation on train dataset
    val_res_train = validate_model_mp(final_model, whole_dataset, model_params=configs.model, train_params=configs.train)
    trues_recon_train = np.concatenate([res["recon"]["true"] for res in val_res_train.values()])
    preds_recon_train = np.concatenate([res["recon"]["pred"] for res in val_res_train.values()])
    trues_y_train = np.concatenate([res["y"]["true"] for res in val_res_train.values()])
    preds_y_train = np.concatenate([res["y"]["pred"] for res in val_res_train.values()])
    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_y_train = perf_eval("TRAIN_SET", trues_y_train, preds_y_train, configs=configs, pred_type=configs.data.pred_type)
    if configs.data.pred_type == "classification":
        eval_y_train, confmat_y_train = eval_y_train
else:
    eval_recon_train = eval_y_train = None

## Evaluation on test dataset
# trues_recon_test, preds_recon_test, trues_y_test, preds_y_test = validate_model(final_model, test_loader, configs.model, configs.train)
val_res_test = validate_model_mp(final_model, test_dataset, model_params=configs.model, train_params=configs.train)
trues_recon_test = np.concatenate([res["recon"]["true"] for res in val_res_test.values()])
preds_recon_test = np.concatenate([res["recon"]["pred"] for res in val_res_test.values()])
trues_y_test = np.concatenate([res["y"]["true"] for res in val_res_test.values()])
preds_y_test = np.concatenate([res["y"]["pred"] for res in val_res_test.values()])
eval_recon_test = perf_eval("TEST_SET", trues_recon_test, preds_recon_test, configs=configs,  pred_type='generative', tokenizer=final_model.module.tokenizer)
eval_y_test = perf_eval("TEST_SET", trues_y_test, preds_y_test, configs=configs, pred_type="classification")
if configs.data.pred_type == "classification":
    eval_y_test, confmat_y_test = eval_y_test

eval_y_res = pd.concat([eval_y_train, eval_y_test], axis=0)
eval_recon_res = pd.concat([eval_recon_train, eval_recon_test], axis=0)

In [ ]:
raise

# (temp) BART experiment

In [ ]:
from transformers import BartTokenizer, BartModel, BartForConditionalGeneration

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# model = BartModel.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
ex_input = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [ ]:
count_parameters(model)

In [ ]:
outputs = model.generate(ex_input["input_ids"], num_beams=2, min_length=0, max_length=20)

In [ ]:
outputs = model(ex_input["input_ids"])

In [ ]:
outputs.logits.argmax(2)